# Max-Cut with QAOA (Qulacs)

Template notebook for solving Max-Cut using Qulacs with a simple
grid-search based QAOA and timing measurement.


In [ ]:
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from qulacs import QuantumState, ParametricQuantumCircuit
from qulacs.observable import create_observable_from_openfermion_text


In [ ]:
n_nodes = 6
G = nx.erdos_renyi_graph(n_nodes, 0.5, seed=1)
pos = nx.spring_layout(G, seed=1)
nx.draw(G, pos, with_labels=True)
plt.title("Max-Cut instance (Qulacs)")
plt.show()


In [ ]:
def maxcut_openfermion_text(G):
    lines = []
    for i, j in G.edges():
        # +1 Z_i Z_j
        lines.append(f"+1.0 Z{i} Z{j}")
    return "\n".join(lines)

ham_str = maxcut_openfermion_text(G)
H = create_observable_from_openfermion_text(ham_str)
print("Hamiltonian from OpenFermion text created.")


In [ ]:
n = len(G.nodes())
reps = 1  # keep depth small for demo

def build_qaoa_circuit(gamma, beta):
    circ = ParametricQuantumCircuit(n)
    # Start in |+> state: H on each qubit
    for i in range(n):
        circ.add_H_gate(i)

    # Cost layer
    for i, j in G.edges():
        circ.add_multi_Pauli_rotation_gate([i, j], [3, 3], 2 * gamma)

    # Mixer layer
    for i in range(n):
        circ.add_X_rotation_gate(i, 2 * beta)

    return circ

def expectation_value(gamma, beta):
    circ = build_qaoa_circuit(gamma, beta)
    state = QuantumState(n)
    state.set_zero_state()
    circ.update_quantum_state(state)
    return H.get_expectation_value(state)


In [ ]:
start = time.perf_counter()

best_E = 1e9
best_params = None

gammas = np.linspace(0, np.pi, 10)
betas = np.linspace(0, np.pi, 10)

for g in gammas:
    for b in betas:
        E = expectation_value(g, b)
        if E < best_E:
            best_E = E
            best_params = (g, b)

end = time.perf_counter()

print("Best energy:", best_E)
print("Best params (gamma, beta):", best_params)
print(f"Runtime (grid search): {end - start:.4f} s")
